In [2]:
# =============================================================================
# WATERMARKING POC: 3 Novel Ideas Comparison
# Fast proof-of-concept for Kaggle P100
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import time
import warnings
warnings.filterwarnings('ignore')

# -----------------------------------------------------------------------------
# SETUP
# -----------------------------------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Device: {device}")
torch.manual_seed(42)
np.random.seed(42)

# -----------------------------------------------------------------------------
# DATA LOADING (Fast - small subset)
# -----------------------------------------------------------------------------
def load_data(n_train=2000, n_test=400):
    train_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')
    test_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv')
    
    train_df = train_df.sample(n=n_train, random_state=42).reset_index(drop=True)
    test_df = test_df.sample(n=n_test, random_state=42).reset_index(drop=True)
    
    X_train = torch.FloatTensor(train_df.drop('label', axis=1).values.reshape(-1, 1, 28, 28) / 255.0)
    y_train = torch.LongTensor(train_df['label'].values)
    X_test = torch.FloatTensor(test_df.drop('label', axis=1).values.reshape(-1, 1, 28, 28) / 255.0)
    y_test = torch.LongTensor(test_df['label'].values)
    
    return X_train, y_train, X_test, y_test

# -----------------------------------------------------------------------------
# BASE CNN
# -----------------------------------------------------------------------------
class BaseCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU()
        )
        self.classifier = nn.Linear(128, 10)
    
    def forward(self, x):
        feat = self.features(x)
        return self.classifier(feat), feat

# =============================================================================
# IDEA 1: ADVERSARIAL SYMBIOSIS WATERMARKING
# Core: Watermark HELPS task performance - removing it hurts accuracy
# =============================================================================

class SymbiosisModel(nn.Module):
    """
    Key Innovation: Watermark features provide USEFUL inductive bias
    - Shared symbiosis layer contributes to BOTH task and watermark
    - Removing watermark = removing useful features = accuracy drops
    """
    def __init__(self):
        super().__init__()
        self.backbone = BaseCNN()
        
        # Secret watermark key
        key = torch.randn(64)
        self.register_buffer('wm_key', F.normalize(key, dim=0))
        
        # SYMBIOSIS LAYER: shared between task and watermark
        # This is the key innovation - features serve dual purpose
        self.symbiosis = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 64)
        )
        
        # Task head uses symbiosis features
        self.task_head = nn.Linear(64, 10)
    
    def forward(self, x, return_all=False):
        _, base_feat = self.backbone(x)
        sym_feat = self.symbiosis(base_feat)
        logits = self.task_head(sym_feat)
        
        if return_all:
            return logits, sym_feat
        return logits
    
    def wm_score(self, x):
        """Watermark detection score"""
        _, sym_feat = self.forward(x, return_all=True)
        sym_feat_norm = F.normalize(sym_feat, dim=1)
        return (sym_feat_norm @ self.wm_key).mean()

def train_symbiosis(model, train_loader, wm_loader, epochs=8):
    """
    Joint training: task + watermark alignment
    Symbiosis = watermark features directly help classification
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(epochs):
        model.train()
        for (x, y), (x_wm, _) in zip(train_loader, wm_loader):
            x, y, x_wm = x.to(device), y.to(device), x_wm.to(device)
            
            # Task loss
            logits = model(x)
            task_loss = F.cross_entropy(logits, y)
            
            # Watermark alignment loss (symbiosis features → key)
            _, sym_feat = model(x_wm, return_all=True)
            sym_feat_norm = F.normalize(sym_feat, dim=1)
            wm_loss = (1 - (sym_feat_norm @ model.wm_key)).mean()
            
            # Combined loss - symbiosis means both are optimized together
            loss = task_loss + 0.5 * wm_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def test_symbiosis_robustness(model, train_loader, wm_loader):
    """Test: Does removing symbiosis hurt accuracy?"""
    
    # Baseline accuracy
    model.eval()
    acc_baseline = evaluate(model, test_loader)
    wm_baseline = model.wm_score(next(iter(wm_loader))[0].to(device)).item()
    
    # ATTACK: Fine-tune to remove watermark
    model_attacked = SymbiosisModel().to(device)
    model_attacked.load_state_dict(model.state_dict())
    
    optimizer = torch.optim.Adam(model_attacked.parameters(), lr=5e-4)
    for _ in range(5):  # 5 epochs attack
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            logits = model_attacked(x)
            loss = F.cross_entropy(logits, y)
            # Try to DISRUPT watermark
            _, sym_feat = model_attacked(x, return_all=True)
            wm_disrupt = (F.normalize(sym_feat, dim=1) @ model_attacked.wm_key).abs().mean()
            loss = loss + 0.3 * wm_disrupt  # Push away from key
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    acc_attacked = evaluate(model_attacked, test_loader)
    wm_attacked = model_attacked.wm_score(next(iter(wm_loader))[0].to(device)).item()
    
    return {
        'acc_baseline': acc_baseline,
        'acc_attacked': acc_attacked,
        'acc_drop': acc_baseline - acc_attacked,  # SYMBIOSIS SUCCESS = positive drop
        'wm_baseline': wm_baseline,
        'wm_attacked': wm_attacked,
    }

# =============================================================================
# IDEA 2: ENTANGLED DUAL WATERMARKS
# Core: Two watermarks A & B where attacking A amplifies B
# =============================================================================

class EntangledModel(nn.Module):
    """
    Key Innovation: Removal trap via entanglement
    - Watermark A and B are in complementary subspaces
    - A's removal gradient AMPLIFIES B's signal
    - Adversary faces impossible tradeoff
    """
    def __init__(self):
        super().__init__()
        self.backbone = BaseCNN()
        
        # Two ORTHOGONAL keys
        key_a = torch.randn(32)
        key_b = torch.randn(32)
        key_a = F.normalize(key_a, dim=0)
        key_b = key_b - (key_b @ key_a) * key_a  # Gram-Schmidt
        key_b = F.normalize(key_b, dim=0)
        
        self.register_buffer('key_A', key_a)
        self.register_buffer('key_B', key_b)
        
        # Dual projection heads
        self.proj_A = nn.Linear(128, 32)
        self.proj_B = nn.Linear(128, 32)
        
        # ENTANGLEMENT: B depends on complement of A
        self.entangle = nn.Linear(32, 32, bias=False)
        
        self.classifier = nn.Linear(128, 10)
    
    def forward(self, x):
        _, feat = self.backbone(x)
        return self.classifier(feat), feat
    
    def get_dual_scores(self, x):
        _, feat = self.backbone(x)
        
        # Projection A
        feat_A = F.normalize(self.proj_A(feat), dim=1)
        score_A = (feat_A @ self.key_A).mean()
        
        # Projection B with ENTANGLEMENT
        feat_B_raw = self.proj_B(feat)
        # Entangle: B receives signal from A's orthogonal complement
        entangled = self.entangle(feat_A.detach())  # Detach creates the trap!
        feat_B = F.normalize(feat_B_raw + 0.7 * entangled, dim=1)
        score_B = (feat_B @ self.key_B).mean()
        
        return score_A, score_B, feat_A, feat_B

def train_entangled(model, train_loader, wm_loader, epochs=8):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(epochs):
        model.train()
        for (x, y), (x_wm, _) in zip(train_loader, wm_loader):
            x, y, x_wm = x.to(device), y.to(device), x_wm.to(device)
            
            # Task
            logits, _ = model(x)
            task_loss = F.cross_entropy(logits, y)
            
            # Dual watermark alignment
            score_A, score_B, _, _ = model.get_dual_scores(x_wm)
            wm_loss = (1 - score_A) + (1 - score_B)
            
            loss = task_loss + 0.3 * wm_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def test_entanglement(model, train_loader, wm_loader):
    """Test: Does attacking A amplify B?"""
    
    model.eval()
    x_wm = next(iter(wm_loader))[0].to(device)
    score_A_before, score_B_before, _, _ = model.get_dual_scores(x_wm)
    score_A_before, score_B_before = score_A_before.item(), score_B_before.item()
    acc_before = evaluate(model, test_loader)
    
    # ATTACK: Specifically target watermark A
    model_attacked = EntangledModel().to(device)
    model_attacked.load_state_dict(model.state_dict())
    
    # Only attack proj_A (adversary targets one watermark)
    optimizer = torch.optim.Adam(model_attacked.proj_A.parameters(), lr=1e-2)
    
    for _ in range(100):  # Aggressive attack on A
        x_wm_batch = next(iter(wm_loader))[0].to(device)
        score_A, _, feat_A, _ = model_attacked.get_dual_scores(x_wm_batch)
        # Push A score toward zero
        loss = score_A.abs()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model_attacked.eval()
    score_A_after, score_B_after, _, _ = model_attacked.get_dual_scores(x_wm)
    score_A_after, score_B_after = score_A_after.item(), score_B_after.item()
    acc_after = evaluate(model_attacked, test_loader)
    
    # ENTANGLEMENT SUCCESS: A dropped but B increased (or survived)
    entanglement_effect = (score_B_after - score_B_before) - (score_A_after - score_A_before)
    
    return {
        'acc_before': acc_before,
        'acc_after': acc_after,
        'score_A_before': score_A_before,
        'score_A_after': score_A_after,
        'score_B_before': score_B_before,
        'score_B_after': score_B_after,
        'entanglement_effect': entanglement_effect,  # Positive = success!
        'A_killed': score_A_after < 0.3,
        'B_survived': score_B_after > 0.5,
    }

# =============================================================================
# IDEA 3: BEHAVIORAL DECISION BOUNDARY WATERMARKING
# Core: Watermark lives in decision boundary SHAPE, not weights
# =============================================================================

class BoundaryModel(nn.Module):
    """
    Key Innovation: Watermark = specific decision boundary geometry
    - Define "probe points" near class boundaries
    - Train model to have specific behavior on probes
    - Survives weight changes if boundary shape preserved
    """
    def __init__(self):
        super().__init__()
        self.backbone = BaseCNN()
        
        # BOUNDARY PROBES: points designed to be near decision boundaries
        # These are our "behavioral watermark"
        probes = []
        targets = []
        for i in range(10):
            # Create probe as mixture of class prototypes (near boundary)
            probe = torch.randn(1, 28, 28) * 0.2
            # Add class-specific pattern
            probe[:, i*2:(i*2)+5, i*2:(i*2)+5] = 1.0
            probes.append(probe)
            targets.append(i)
        
        self.register_buffer('probes', torch.stack(probes))
        self.register_buffer('probe_targets', torch.LongTensor(targets))
        
        # Store original boundary signature
        self.register_buffer('original_signature', torch.zeros(10, 10))
    
    def forward(self, x):
        logits, feat = self.backbone(x)
        return logits, feat
    
    def get_boundary_signature(self):
        """Get probability distribution on probes = behavioral fingerprint"""
        self.eval()
        with torch.no_grad():
            logits, _ = self.backbone(self.probes.to(device))
            return F.softmax(logits, dim=1)
    
    def verify_ownership(self, threshold=0.7):
        """Verify by comparing current vs original boundary behavior"""
        current_sig = self.get_boundary_signature()
        similarity = F.cosine_similarity(
            self.original_signature.flatten().unsqueeze(0),
            current_sig.flatten().unsqueeze(0)
        )
        return similarity.item(), similarity.item() > threshold

def train_boundary(model, train_loader, epochs=8):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            # Task loss
            logits, _ = model(x)
            task_loss = F.cross_entropy(logits, y)
            
            # BOUNDARY WATERMARK: probes must predict specific targets
            probe_logits, _ = model.backbone(model.probes.to(device))
            boundary_loss = F.cross_entropy(probe_logits, model.probe_targets.to(device))
            
            loss = task_loss + 0.5 * boundary_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Store signature after training
    model.original_signature = model.get_boundary_signature().clone()

def test_boundary_robustness(model, train_loader):
    """Test: Does boundary signature survive attacks?"""
    
    model.eval()
    sig_before, _ = model.verify_ownership()
    acc_before = evaluate(model, test_loader)
    
    # Check probe accuracy
    with torch.no_grad():
        probe_logits, _ = model.backbone(model.probes.to(device))
        probe_preds = probe_logits.argmax(dim=1)
        probe_acc_before = (probe_preds == model.probe_targets.to(device)).float().mean().item()
    
    # ATTACK: Standard fine-tuning (unaware of boundary watermark)
    model_attacked = BoundaryModel().to(device)
    model_attacked.load_state_dict(model.state_dict())
    
    optimizer = torch.optim.Adam(model_attacked.parameters(), lr=5e-4)
    for _ in range(5):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            logits, _ = model_attacked(x)
            loss = F.cross_entropy(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    model_attacked.original_signature = model.original_signature.clone()
    sig_after, verified = model_attacked.verify_ownership()
    acc_after = evaluate(model_attacked, test_loader)
    
    with torch.no_grad():
        probe_logits, _ = model_attacked.backbone(model_attacked.probes.to(device))
        probe_preds = probe_logits.argmax(dim=1)
        probe_acc_after = (probe_preds == model_attacked.probe_targets.to(device)).float().mean().item()
    
    return {
        'acc_before': acc_before,
        'acc_after': acc_after,
        'signature_before': sig_before,
        'signature_after': sig_after,
        'verified_after_attack': verified,
        'probe_acc_before': probe_acc_before,
        'probe_acc_after': probe_acc_after,
    }

# =============================================================================
# EVALUATION HELPER
# =============================================================================

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x) if not isinstance(model(x), tuple) else model(x)[0]
            correct += (logits.argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

# =============================================================================
# MAIN EXPERIMENT
# =============================================================================

print("\n" + "="*70)
print("   🔬 NOVEL WATERMARKING IDEAS - PROOF OF CONCEPT")
print("="*70)

# Load data
X_train, y_train, X_test, y_test = load_data(n_train=2000, n_test=400)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=64, pin_memory=True)

# Watermark subset (10%)
wm_idx = np.random.choice(len(train_ds), len(train_ds)//10, replace=False)
wm_ds = TensorDataset(X_train[wm_idx], y_train[wm_idx])
wm_loader = DataLoader(wm_ds, batch_size=64, shuffle=True, pin_memory=True)

results = {}

# -----------------------------------------------------------------------------
# TEST IDEA 1: SYMBIOSIS
# -----------------------------------------------------------------------------
print("\n" + "-"*70)
print("💫 IDEA 1: ADVERSARIAL SYMBIOSIS WATERMARKING")
print("-"*70)
print("Concept: Watermark features HELP task → removing hurts accuracy")

t0 = time.time()
sym_model = SymbiosisModel().to(device)
train_symbiosis(sym_model, train_loader, wm_loader, epochs=8)
sym_results = test_symbiosis_robustness(sym_model, train_loader, wm_loader)
sym_results['time'] = time.time() - t0

print(f"\n  Accuracy:    {sym_results['acc_baseline']:.1%} → {sym_results['acc_attacked']:.1%}")
print(f"  Acc Drop:    {sym_results['acc_drop']:.1%} (positive = symbiosis works!)")
print(f"  WM Score:    {sym_results['wm_baseline']:.3f} → {sym_results['wm_attacked']:.3f}")
print(f"  Time:        {sym_results['time']:.1f}s")

symbiosis_success = sym_results['acc_drop'] > 0.02  # Acc dropped when attacking WM
print(f"\n  ✅ SYMBIOSIS EFFECT: {'CONFIRMED' if symbiosis_success else 'NOT CONFIRMED'}")

results['Symbiosis'] = sym_results

# -----------------------------------------------------------------------------
# TEST IDEA 2: ENTANGLED DUAL
# -----------------------------------------------------------------------------
print("\n" + "-"*70)
print("🔗 IDEA 2: ENTANGLED DUAL WATERMARKS")
print("-"*70)
print("Concept: Attacking watermark A amplifies watermark B (removal trap)")

t0 = time.time()
ent_model = EntangledModel().to(device)
train_entangled(ent_model, train_loader, wm_loader, epochs=8)
ent_results = test_entanglement(ent_model, train_loader, wm_loader)
ent_results['time'] = time.time() - t0

print(f"\n  Accuracy:    {ent_results['acc_before']:.1%} → {ent_results['acc_after']:.1%}")
print(f"  Score A:     {ent_results['score_A_before']:.3f} → {ent_results['score_A_after']:.3f}")
print(f"  Score B:     {ent_results['score_B_before']:.3f} → {ent_results['score_B_after']:.3f}")
print(f"  Entangle Δ:  {ent_results['entanglement_effect']:.3f} (positive = trap works!)")
print(f"  Time:        {ent_results['time']:.1f}s")

entangle_success = ent_results['entanglement_effect'] > 0 or ent_results['B_survived']
print(f"\n  ✅ ENTANGLEMENT TRAP: {'TRIGGERED' if entangle_success else 'NOT TRIGGERED'}")
print(f"     A killed: {ent_results['A_killed']}, B survived: {ent_results['B_survived']}")

results['Entangled'] = ent_results

# -----------------------------------------------------------------------------
# TEST IDEA 3: BOUNDARY WATERMARK
# -----------------------------------------------------------------------------
print("\n" + "-"*70)
print("📐 IDEA 3: BEHAVIORAL DECISION BOUNDARY WATERMARKING")
print("-"*70)
print("Concept: Watermark in boundary SHAPE, survives weight changes")

t0 = time.time()
bnd_model = BoundaryModel().to(device)
train_boundary(bnd_model, train_loader, epochs=8)
bnd_results = test_boundary_robustness(bnd_model, train_loader)
bnd_results['time'] = time.time() - t0

print(f"\n  Accuracy:    {bnd_results['acc_before']:.1%} → {bnd_results['acc_after']:.1%}")
print(f"  Signature:   {bnd_results['signature_before']:.3f} → {bnd_results['signature_after']:.3f}")
print(f"  Probe Acc:   {bnd_results['probe_acc_before']:.1%} → {bnd_results['probe_acc_after']:.1%}")
print(f"  Verified:    {bnd_results['verified_after_attack']}")
print(f"  Time:        {bnd_results['time']:.1f}s")

boundary_success = bnd_results['verified_after_attack']
print(f"\n  ✅ BOUNDARY WATERMARK: {'SURVIVED' if boundary_success else 'REMOVED'}")

results['Boundary'] = bnd_results

# =============================================================================
# FINAL COMPARISON
# =============================================================================

print("\n" + "="*70)
print("   📊 FINAL COMPARISON")
print("="*70)

print("\n{:<18} {:<12} {:<12} {:<18} {:<8}".format(
    "Method", "Acc Before", "Acc After", "Key Metric", "Status"
))
print("-"*70)

# Symbiosis
status = "✅ WORKS" if symbiosis_success else "❌ WEAK"
print("{:<18} {:<12.1%} {:<12.1%} {:<18} {:<8}".format(
    "Symbiosis", 
    results['Symbiosis']['acc_baseline'],
    results['Symbiosis']['acc_attacked'],
    f"AccDrop={results['Symbiosis']['acc_drop']:.1%}",
    status
))

# Entangled
status = "✅ WORKS" if entangle_success else "❌ WEAK"
print("{:<18} {:<12.1%} {:<12.1%} {:<18} {:<8}".format(
    "Entangled Dual",
    results['Entangled']['acc_before'],
    results['Entangled']['acc_after'],
    f"Entangle={results['Entangled']['entanglement_effect']:.2f}",
    status
))

# Boundary
status = "✅ WORKS" if boundary_success else "❌ WEAK"
print("{:<18} {:<12.1%} {:<12.1%} {:<18} {:<8}".format(
    "Boundary",
    results['Boundary']['acc_before'],
    results['Boundary']['acc_after'],
    f"Sig={results['Boundary']['signature_after']:.2f}",
    status
))

# =============================================================================
# WINNER DETERMINATION
# =============================================================================

print("\n" + "="*70)
print("   🏆 VERDICT")
print("="*70)

scores = {}

# Symbiosis score: acc drop (more = better) + wm survival
scores['Symbiosis'] = (
    results['Symbiosis']['acc_drop'] * 10 +  # Weight accuracy drop highly
    results['Symbiosis']['wm_attacked']
)

# Entangled score: entanglement effect + B survival
scores['Entangled'] = (
    results['Entangled']['entanglement_effect'] * 2 +
    results['Entangled']['score_B_after'] +
    (1.0 if results['Entangled']['B_survived'] else 0.0)
)

# Boundary score: signature retention + verification
scores['Boundary'] = (
    results['Boundary']['signature_after'] +
    (1.0 if results['Boundary']['verified_after_attack'] else 0.0) +
    results['Boundary']['probe_acc_after']
)

print(f"\nScores:")
for name, score in sorted(scores.items(), key=lambda x: -x[1]):
    print(f"  {name}: {score:.3f}")

winner = max(scores, key=scores.get)
print(f"\n{'='*70}")
print(f"   🥇 BEST METHOD: {winner.upper()}")
print(f"{'='*70}")

print(f"""
📋 RECOMMENDATIONS:
""")

if winner == "Symbiosis":
    print("""
   ADVERSARIAL SYMBIOSIS is most promising because:
   - Removing watermark HURTS accuracy (novel defense!)
   - Creates mutual benefit between task and watermark
   - Adversary faces impossible tradeoff
   
   Next steps:
   - Strengthen symbiosis coupling
   - Test on larger models (ResNet, ViT)
   - Measure symbiosis strength formally
""")
elif winner == "Entangled":
    print("""
   ENTANGLED DUAL WATERMARKS is most promising because:
   - Creates removal trap (attack A → B gets stronger)
   - Two layers of protection
   - Novel defense mechanism
   
   Next steps:
   - Strengthen entanglement coupling
   - Add more watermark pairs (A, B, C, ...)
   - Mathematical proof of entanglement
""")
else:
    print("""
   BOUNDARY WATERMARKING is most promising because:
   - Survives weight modifications
   - Behavioral (functional) not structural
   - Hard to remove without changing model behavior
   
   Next steps:
   - Design better boundary probes
   - Test robustness to pruning/quantization
   - Formal verification of boundary preservation
""")

print("\n✅ POC Complete! Ready for full implementation.")

🚀 Device: cuda

   🔬 NOVEL WATERMARKING IDEAS - PROOF OF CONCEPT

----------------------------------------------------------------------
💫 IDEA 1: ADVERSARIAL SYMBIOSIS WATERMARKING
----------------------------------------------------------------------
Concept: Watermark features HELP task → removing hurts accuracy

  Accuracy:    47.2% → 73.5%
  Acc Drop:    -26.2% (positive = symbiosis works!)
  WM Score:    0.874 → 0.341
  Time:        4.8s

  ✅ SYMBIOSIS EFFECT: NOT CONFIRMED

----------------------------------------------------------------------
🔗 IDEA 2: ENTANGLED DUAL WATERMARKS
----------------------------------------------------------------------
Concept: Attacking watermark A amplifies watermark B (removal trap)

  Accuracy:    52.8% → 52.8%
  Score A:     0.985 → 0.012
  Score B:     0.981 → 0.982
  Entangle Δ:  0.974 (positive = trap works!)
  Time:        0.6s

  ✅ ENTANGLEMENT TRAP: TRIGGERED
     A killed: True, B survived: True

-----------------------------------------

In [4]:
# =============================================================================
# ENTANGLED DUAL WATERMARKS: COMPREHENSIVE EXPERIMENTAL STUDY
# For Q1 Journal Submission
# =============================================================================
# 
# Title: Entangled Dual Neural Network Watermarking: A Removal-Resistant 
#        Ownership Verification Framework via Orthogonal Subspace Coupling
#
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, Subset
from collections import defaultdict
from scipy import stats
import time
import warnings
import math
from datetime import datetime
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================

class Config:
    # Reproducibility
    SEEDS = [42, 2024, 7, 123, 999]
    
    # Data
    N_TRAIN = 1000
    N_TEST = 200
    WATERMARK_RATIO = 0.10
    
    # Model configurations
    MODEL_CONFIGS = {
        'Small': {'channels': [16, 32], 'fc_dim': 64},
        'Medium': {'channels': [32, 64], 'fc_dim': 128},
        'Large': {'channels': [64, 128], 'fc_dim': 256},
    }
    
    # Training
    BATCH_SIZE = 64
    EPOCHS_TASK = 15
    EPOCHS_WATERMARK = 10
    LR_TASK = 1e-3
    LR_WATERMARK = 1e-3
    
    # Watermark
    KEY_DIM = 64
    ENTANGLE_STRENGTH = 0.7
    MARGIN = 0.1
    
    # Attack configurations
    ATTACK_EPOCHS = [1, 3, 5, 10]
    PRUNING_RATIOS = [0.1, 0.3, 0.5, 0.7]
    FINETUNE_LRS = [1e-4, 5e-4, 1e-3]
    
    # Ablation
    ENTANGLE_STRENGTHS = [0.0, 0.3, 0.5, 0.7, 1.0]
    KEY_DIMS = [32, 64, 128, 256]
    WM_RATIOS = [0.05, 0.10, 0.15, 0.20]

# =============================================================================
# SETUP & UTILITIES
# =============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def print_header(title, char='=', width=80):
    print(f"\n{char*width}")
    print(f"  {title}")
    print(f"{char*width}")

def print_subheader(title, char='-', width=80):
    print(f"\n{char*width}")
    print(f"  {title}")
    print(f"{char*width}")

def format_pvalue(p):
    if p < 0.001:
        return "<0.001***"
    elif p < 0.01:
        return f"{p:.4f}**"
    elif p < 0.05:
        return f"{p:.4f}*"
    else:
        return f"{p:.4f}"

def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (np.mean(group1) - np.mean(group2)) / (pooled_std + 1e-8)

# =============================================================================
# DATA LOADING
# =============================================================================

def load_fashion_mnist(n_train, n_test, seed=42):
    np.random.seed(seed)
    
    train_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')
    test_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv')
    
    train_df = train_df.sample(n=min(n_train, len(train_df)), random_state=seed).reset_index(drop=True)
    test_df = test_df.sample(n=min(n_test, len(test_df)), random_state=seed).reset_index(drop=True)
    
    X_train = torch.FloatTensor(train_df.drop('label', axis=1).values.reshape(-1, 1, 28, 28) / 255.0)
    y_train = torch.LongTensor(train_df['label'].values)
    X_test = torch.FloatTensor(test_df.drop('label', axis=1).values.reshape(-1, 1, 28, 28) / 255.0)
    y_test = torch.LongTensor(test_df['label'].values)
    
    return X_train, y_train, X_test, y_test

# =============================================================================
# MODEL ARCHITECTURES
# =============================================================================

class FlexibleCNN(nn.Module):
    """Configurable CNN backbone for different model sizes"""
    def __init__(self, channels=[32, 64], fc_dim=128):
        super().__init__()
        
        layers = []
        in_ch = 1
        for ch in channels:
            layers.extend([
                nn.Conv2d(in_ch, ch, 3, padding=1),
                nn.BatchNorm2d(ch),
                nn.ReLU(),
                nn.MaxPool2d(2)
            ])
            in_ch = ch
        
        self.features = nn.Sequential(*layers)
        
        # Calculate feature map size
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 28, 28)
            feat_size = self.features(dummy).view(1, -1).size(1)
        
        self.fc = nn.Sequential(
            nn.Linear(feat_size, fc_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.fc_dim = fc_dim
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class EntangledDualWatermarkModel(nn.Module):
    """
    ENTANGLED DUAL WATERMARKING MODEL
    
    Key Innovation: Two watermarks A and B exist in orthogonal subspaces,
    coupled through an entanglement mechanism such that attacking A
    amplifies or preserves B's signal.
    
    Mathematical Foundation:
    - Keys k_A, k_B are orthogonal: <k_A, k_B> = 0
    - Projections P_A, P_B map features to respective subspaces
    - Entanglement: P_B receives signal from orthogonal complement of P_A
    - This creates a "removal trap": attacking A inadvertently strengthens B
    """
    
    def __init__(self, backbone_config, key_dim=64, entangle_strength=0.7, seed=42):
        super().__init__()
        
        set_seed(seed)
        
        # Backbone
        self.backbone = FlexibleCNN(**backbone_config)
        feat_dim = self.backbone.fc_dim
        
        # Task classifier
        self.classifier = nn.Sequential(
            nn.Linear(feat_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
        # Generate ORTHOGONAL keys via Gram-Schmidt
        key_a = torch.randn(key_dim)
        key_a = F.normalize(key_a, dim=0)
        
        key_b = torch.randn(key_dim)
        key_b = key_b - (key_b @ key_a) * key_a  # Orthogonalize
        key_b = F.normalize(key_b, dim=0)
        
        self.register_buffer('key_A', key_a)
        self.register_buffer('key_B', key_b)
        
        # Verify orthogonality
        ortho_check = (key_a @ key_b).abs().item()
        assert ortho_check < 1e-5, f"Keys not orthogonal: {ortho_check}"
        
        # Projection heads for dual watermarks
        self.proj_A = nn.Sequential(
            nn.Linear(feat_dim, key_dim * 2),
            nn.ReLU(),
            nn.Linear(key_dim * 2, key_dim)
        )
        
        self.proj_B = nn.Sequential(
            nn.Linear(feat_dim, key_dim * 2),
            nn.ReLU(),
            nn.Linear(key_dim * 2, key_dim)
        )
        
        # ENTANGLEMENT MECHANISM
        # This linear layer couples A's orthogonal complement to B
        self.entangle = nn.Linear(key_dim, key_dim, bias=False)
        nn.init.orthogonal_(self.entangle.weight)
        
        self.entangle_strength = entangle_strength
        self.key_dim = key_dim
        
    def forward(self, x):
        feat = self.backbone(x)
        logits = self.classifier(feat)
        return logits
    
    def get_features(self, x):
        return self.backbone(x)
    
    def compute_watermark_scores(self, x, return_features=False):
        """
        Compute dual watermark detection scores.
        
        Returns:
            score_A: Cosine similarity between proj_A(features) and key_A
            score_B: Cosine similarity between entangled_proj_B(features) and key_B
        """
        feat = self.backbone(x)
        
        # Watermark A projection
        feat_A = self.proj_A(feat)
        feat_A_norm = F.normalize(feat_A, dim=1)
        score_A = feat_A_norm @ self.key_A
        
        # Watermark B projection with ENTANGLEMENT
        feat_B_raw = self.proj_B(feat)
        
        # Entanglement: B receives transformed signal from A
        # The .detach() is crucial - it creates the asymmetric trap
        entangled_signal = self.entangle(feat_A_norm.detach())
        feat_B_entangled = feat_B_raw + self.entangle_strength * entangled_signal
        feat_B_norm = F.normalize(feat_B_entangled, dim=1)
        score_B = feat_B_norm @ self.key_B
        
        if return_features:
            return score_A, score_B, feat_A_norm, feat_B_norm
        return score_A, score_B
    
    def get_orthogonality_metrics(self):
        """Compute orthogonality metrics for theoretical validation"""
        key_ortho = (self.key_A @ self.key_B).abs().item()
        
        # Check projection head independence
        with torch.no_grad():
            dummy = torch.randn(100, self.backbone.fc_dim).to(next(self.parameters()).device)
            feat_A = F.normalize(self.proj_A(dummy), dim=1)
            feat_B = F.normalize(self.proj_B(dummy), dim=1)
            proj_correlation = (feat_A * feat_B).sum(dim=1).abs().mean().item()
        
        return {
            'key_orthogonality': key_ortho,
            'projection_correlation': proj_correlation
        }

# =============================================================================
# TRAINING PROCEDURES
# =============================================================================

def train_task(model, train_loader, epochs, lr, verbose=False):
    """Phase 1: Standard task training"""
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    model.train()
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        total_loss, correct, total = 0, 0, 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            correct += (logits.argmax(1) == y).sum().item()
            total += y.size(0)
        
        scheduler.step()
        history['loss'].append(total_loss / len(train_loader))
        history['acc'].append(correct / total)
        
        if verbose and (epoch + 1) % 5 == 0:
            print(f"    Epoch {epoch+1}/{epochs}: Loss={history['loss'][-1]:.4f}, Acc={history['acc'][-1]:.4f}")
    
    return history

def train_watermark(model, train_loader, wm_loader, epochs, lr, margin=0.1, verbose=False):
    """Phase 2: Dual watermark embedding with entanglement"""
    
    # Freeze backbone and classifier
    for param in model.backbone.parameters():
        param.requires_grad = False
    for param in model.classifier.parameters():
        param.requires_grad = False
    
    # Only train watermark components
    wm_params = list(model.proj_A.parameters()) + list(model.proj_B.parameters()) + list(model.entangle.parameters())
    optimizer = torch.optim.Adam(wm_params, lr=lr)
    
    model.train()
    history = {'loss': [], 'score_A': [], 'score_B': []}
    
    for epoch in range(epochs):
        total_loss = 0
        all_scores_A, all_scores_B = [], []
        
        wm_iter = iter(wm_loader)
        
        for x_clean, _ in train_loader:
            try:
                x_wm, _ = next(wm_iter)
            except StopIteration:
                wm_iter = iter(wm_loader)
                x_wm, _ = next(wm_iter)
            
            x_clean, x_wm = x_clean.to(device), x_wm.to(device)
            
            # Watermarked samples should align with keys
            score_A_wm, score_B_wm = model.compute_watermark_scores(x_wm)
            loss_wm = (1 - score_A_wm).mean() + (1 - score_B_wm).mean()
            
            # Clean samples should project to null space (away from keys)
            score_A_clean, score_B_clean = model.compute_watermark_scores(x_clean)
            loss_clean = (
                F.relu(score_A_clean + margin).mean() + 
                F.relu(score_B_clean + margin).mean()
            )
            
            loss = loss_wm + loss_clean
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            all_scores_A.extend(score_A_wm.detach().cpu().numpy())
            all_scores_B.extend(score_B_wm.detach().cpu().numpy())
        
        history['loss'].append(total_loss / len(train_loader))
        history['score_A'].append(np.mean(all_scores_A))
        history['score_B'].append(np.mean(all_scores_B))
        
        if verbose and (epoch + 1) % 3 == 0:
            print(f"    Epoch {epoch+1}/{epochs}: Loss={history['loss'][-1]:.4f}, "
                  f"A={history['score_A'][-1]:.4f}, B={history['score_B'][-1]:.4f}")
    
    # Unfreeze for potential attacks
    for param in model.backbone.parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True
    
    return history

# =============================================================================
# EVALUATION FUNCTIONS
# =============================================================================

def evaluate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            correct += (logits.argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

def evaluate_watermark(model, clean_loader, wm_loader):
    """Comprehensive watermark evaluation"""
    model.eval()
    
    results = {
        'wm_scores_A': [], 'wm_scores_B': [],
        'clean_scores_A': [], 'clean_scores_B': []
    }
    
    with torch.no_grad():
        # Watermarked samples
        for x, _ in wm_loader:
            x = x.to(device)
            score_A, score_B = model.compute_watermark_scores(x)
            results['wm_scores_A'].extend(score_A.cpu().numpy())
            results['wm_scores_B'].extend(score_B.cpu().numpy())
        
        # Clean samples
        for x, _ in clean_loader:
            x = x.to(device)
            score_A, score_B = model.compute_watermark_scores(x)
            results['clean_scores_A'].extend(score_A.cpu().numpy())
            results['clean_scores_B'].extend(score_B.cpu().numpy())
    
    # Compute statistics
    metrics = {}
    for key in ['A', 'B']:
        wm = np.array(results[f'wm_scores_{key}'])
        clean = np.array(results[f'clean_scores_{key}'])
        
        metrics[f'wm_mean_{key}'] = np.mean(wm)
        metrics[f'wm_std_{key}'] = np.std(wm)
        metrics[f'clean_mean_{key}'] = np.mean(clean)
        metrics[f'clean_std_{key}'] = np.std(clean)
        metrics[f'separation_{key}'] = np.mean(wm) - np.mean(clean)
        metrics[f'effect_size_{key}'] = cohens_d(wm, clean)
        
        # Statistical test
        t_stat, p_value = stats.ttest_ind(wm, clean)
        metrics[f't_stat_{key}'] = t_stat
        metrics[f'p_value_{key}'] = p_value
        
        # Detection rate at threshold 0
        metrics[f'detection_rate_{key}'] = (wm > 0).mean()
        metrics[f'false_positive_{key}'] = (clean > 0).mean()
    
    return metrics, results

# =============================================================================
# ATTACK IMPLEMENTATIONS
# =============================================================================

def attack_finetune(model, train_loader, epochs, lr):
    """Standard fine-tuning attack"""
    model_copy = type(model)(
        model.backbone.__class__.__dict__['__init__'].__code__.co_varnames,
        model.key_dim,
        model.entangle_strength
    )
    # Deep copy weights
    model_copy = EntangledDualWatermarkModel(
        {'channels': [c.out_channels for c in model.backbone.features if isinstance(c, nn.Conv2d)],
         'fc_dim': model.backbone.fc_dim},
        model.key_dim,
        model.entangle_strength
    ).to(device)
    model_copy.load_state_dict(model.state_dict())
    
    optimizer = torch.optim.SGD(model_copy.parameters(), lr=lr, momentum=0.9)
    model_copy.train()
    
    for _ in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = F.cross_entropy(model_copy(x), y)
            loss.backward()
            optimizer.step()
    
    return model_copy

def attack_finetune_simple(model, train_loader, epochs, lr):
    """Simple fine-tuning attack that modifies model in place"""
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    model.train()
    
    for _ in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = F.cross_entropy(model(x), y)
            loss.backward()
            optimizer.step()
    
    return model

def attack_targeted_A(model, wm_loader, iterations=200, lr=0.01):
    """Targeted attack: specifically try to remove watermark A"""
    # Only attack proj_A
    optimizer = torch.optim.Adam(model.proj_A.parameters(), lr=lr)
    model.train()
    
    for _ in range(iterations):
        for x, _ in wm_loader:
            x = x.to(device)
            score_A, _ = model.compute_watermark_scores(x)
            # Push A score toward zero
            loss = score_A.abs().mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break  # One batch per iteration
    
    return model

def attack_targeted_B(model, wm_loader, iterations=200, lr=0.01):
    """Targeted attack: specifically try to remove watermark B"""
    optimizer = torch.optim.Adam(model.proj_B.parameters(), lr=lr)
    model.train()
    
    for _ in range(iterations):
        for x, _ in wm_loader:
            x = x.to(device)
            _, score_B = model.compute_watermark_scores(x)
            loss = score_B.abs().mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
    
    return model

def attack_pruning(model, ratio):
    """Magnitude-based weight pruning attack"""
    import copy
    model_pruned = copy.deepcopy(model)
    
    # Collect all weights
    all_weights = []
    for name, param in model_pruned.named_parameters():
        if 'weight' in name and param.dim() > 1:
            all_weights.append(param.data.abs().view(-1))
    
    all_weights = torch.cat(all_weights)
    threshold = torch.quantile(all_weights, ratio)
    
    # Prune weights below threshold
    for name, param in model_pruned.named_parameters():
        if 'weight' in name and param.dim() > 1:
            mask = param.data.abs() > threshold
            param.data *= mask.float()
    
    return model_pruned

def attack_dual(model, wm_loader, iterations=200, lr=0.01):
    """Attack both watermarks simultaneously"""
    params = list(model.proj_A.parameters()) + list(model.proj_B.parameters())
    optimizer = torch.optim.Adam(params, lr=lr)
    model.train()
    
    for _ in range(iterations):
        for x, _ in wm_loader:
            x = x.to(device)
            score_A, score_B = model.compute_watermark_scores(x)
            loss = score_A.abs().mean() + score_B.abs().mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
    
    return model

# =============================================================================
# COMPREHENSIVE EXPERIMENT RUNNER
# =============================================================================

def run_single_experiment(config, model_name, seed, verbose=False):
    """Run complete experiment for one configuration"""
    set_seed(seed)
    
    # Load data
    X_train, y_train, X_test, y_test = load_fashion_mnist(
        config.N_TRAIN, config.N_TEST, seed
    )
    
    # Create data loaders
    train_ds = TensorDataset(X_train, y_train)
    test_ds = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE, pin_memory=True)
    
    # Create watermark subset
    n_wm = int(len(train_ds) * config.WATERMARK_RATIO)
    wm_indices = np.random.choice(len(train_ds), n_wm, replace=False)
    clean_indices = np.array([i for i in range(len(train_ds)) if i not in wm_indices])
    
    wm_ds = Subset(train_ds, wm_indices)
    clean_ds = Subset(train_ds, clean_indices[:n_wm])  # Same size for fair comparison
    
    wm_loader = DataLoader(wm_ds, batch_size=config.BATCH_SIZE, shuffle=True, pin_memory=True)
    clean_loader = DataLoader(clean_ds, batch_size=config.BATCH_SIZE, shuffle=True, pin_memory=True)
    
    # Initialize model
    model = EntangledDualWatermarkModel(
        config.MODEL_CONFIGS[model_name],
        key_dim=config.KEY_DIM,
        entangle_strength=config.ENTANGLE_STRENGTH,
        seed=seed
    ).to(device)
    
    results = {'config': model_name, 'seed': seed}
    
    # Phase 1: Task training
    if verbose:
        print(f"  Training task...")
    task_history = train_task(model, train_loader, config.EPOCHS_TASK, config.LR_TASK, verbose)
    results['task_acc_after_phase1'] = evaluate_accuracy(model, test_loader)
    
    # Phase 2: Watermark embedding
    if verbose:
        print(f"  Embedding watermarks...")
    wm_history = train_watermark(model, train_loader, wm_loader, config.EPOCHS_WATERMARK, 
                                  config.LR_WATERMARK, config.MARGIN, verbose)
    
    # Baseline evaluation
    results['task_acc_baseline'] = evaluate_accuracy(model, test_loader)
    wm_metrics, wm_raw = evaluate_watermark(model, clean_loader, wm_loader)
    results.update({f'baseline_{k}': v for k, v in wm_metrics.items()})
    
    # Orthogonality metrics
    ortho = model.get_orthogonality_metrics()
    results.update({f'ortho_{k}': v for k, v in ortho.items()})
    
    # Store model state for attacks
    import copy
    baseline_state = copy.deepcopy(model.state_dict())
    
    # =========================================================================
    # ATTACK EXPERIMENTS
    # =========================================================================
    
    # Attack 1: Targeted attack on A (CRITICAL - tests entanglement)
    if verbose:
        print(f"  Attack: Targeted A...")
    model.load_state_dict(copy.deepcopy(baseline_state))
    model = attack_targeted_A(model, wm_loader, iterations=200, lr=0.01)
    results['attack_A_acc'] = evaluate_accuracy(model, test_loader)
    wm_metrics_A, _ = evaluate_watermark(model, clean_loader, wm_loader)
    results.update({f'attack_A_{k}': v for k, v in wm_metrics_A.items()})
    
    # ENTANGLEMENT EFFECT: Change in B when A is attacked
    results['entanglement_effect_A'] = (
        results['attack_A_wm_mean_B'] - results['baseline_wm_mean_B']
    ) - (
        results['attack_A_wm_mean_A'] - results['baseline_wm_mean_A']
    )
    
    # Attack 2: Targeted attack on B
    if verbose:
        print(f"  Attack: Targeted B...")
    model.load_state_dict(copy.deepcopy(baseline_state))
    model = attack_targeted_B(model, wm_loader, iterations=200, lr=0.01)
    results['attack_B_acc'] = evaluate_accuracy(model, test_loader)
    wm_metrics_B, _ = evaluate_watermark(model, clean_loader, wm_loader)
    results.update({f'attack_B_{k}': v for k, v in wm_metrics_B.items()})
    
    results['entanglement_effect_B'] = (
        results['attack_B_wm_mean_A'] - results['baseline_wm_mean_A']
    ) - (
        results['attack_B_wm_mean_B'] - results['baseline_wm_mean_B']
    )
    
    # Attack 3: Dual attack (both A and B)
    if verbose:
        print(f"  Attack: Dual...")
    model.load_state_dict(copy.deepcopy(baseline_state))
    model = attack_dual(model, wm_loader, iterations=200, lr=0.01)
    results['attack_dual_acc'] = evaluate_accuracy(model, test_loader)
    wm_metrics_dual, _ = evaluate_watermark(model, clean_loader, wm_loader)
    results.update({f'attack_dual_{k}': v for k, v in wm_metrics_dual.items()})
    
    # Attack 4: Fine-tuning attacks at different intensities
    for ft_epochs in [3, 5, 10]:
        if verbose:
            print(f"  Attack: Fine-tune {ft_epochs} epochs...")
        model.load_state_dict(copy.deepcopy(baseline_state))
        model = attack_finetune_simple(model, train_loader, ft_epochs, 1e-3)
        results[f'attack_ft{ft_epochs}_acc'] = evaluate_accuracy(model, test_loader)
        wm_metrics_ft, _ = evaluate_watermark(model, clean_loader, wm_loader)
        results.update({f'attack_ft{ft_epochs}_{k}': v for k, v in wm_metrics_ft.items()})
    
    # Attack 5: Pruning attacks
    for prune_ratio in [0.3, 0.5, 0.7]:
        if verbose:
            print(f"  Attack: Prune {int(prune_ratio*100)}%...")
        model.load_state_dict(copy.deepcopy(baseline_state))
        model_pruned = attack_pruning(model, prune_ratio)
        results[f'attack_prune{int(prune_ratio*100)}_acc'] = evaluate_accuracy(model_pruned, test_loader)
        wm_metrics_prune, _ = evaluate_watermark(model_pruned, clean_loader, wm_loader)
        results.update({f'attack_prune{int(prune_ratio*100)}_{k}': v for k, v in wm_metrics_prune.items()})
    
    return results

# =============================================================================
# ABLATION STUDIES
# =============================================================================

def run_ablation_entangle_strength(config, seeds=[42, 2024]):
    """Ablation: Effect of entanglement strength"""
    results = []
    
    for strength in config.ENTANGLE_STRENGTHS:
        for seed in seeds:
            set_seed(seed)
            
            X_train, y_train, X_test, y_test = load_fashion_mnist(
                config.N_TRAIN, config.N_TEST, seed
            )
            
            train_ds = TensorDataset(X_train, y_train)
            test_ds = TensorDataset(X_test, y_test)
            train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
            test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE)
            
            n_wm = int(len(train_ds) * config.WATERMARK_RATIO)
            wm_indices = np.random.choice(len(train_ds), n_wm, replace=False)
            clean_indices = [i for i in range(len(train_ds)) if i not in wm_indices]
            
            wm_ds = Subset(train_ds, wm_indices)
            clean_ds = Subset(train_ds, clean_indices[:n_wm])
            wm_loader = DataLoader(wm_ds, batch_size=config.BATCH_SIZE, shuffle=True)
            clean_loader = DataLoader(clean_ds, batch_size=config.BATCH_SIZE)
            
            model = EntangledDualWatermarkModel(
                config.MODEL_CONFIGS['Medium'],
                key_dim=config.KEY_DIM,
                entangle_strength=strength,
                seed=seed
            ).to(device)
            
            train_task(model, train_loader, config.EPOCHS_TASK, config.LR_TASK)
            train_watermark(model, train_loader, wm_loader, config.EPOCHS_WATERMARK, config.LR_WATERMARK)
            
            acc = evaluate_accuracy(model, test_loader)
            wm_metrics, _ = evaluate_watermark(model, clean_loader, wm_loader)
            
            # Test entanglement
            import copy
            baseline_state = copy.deepcopy(model.state_dict())
            model = attack_targeted_A(model, wm_loader, iterations=200, lr=0.01)
            wm_after, _ = evaluate_watermark(model, clean_loader, wm_loader)
            
            entangle_effect = (wm_after['wm_mean_B'] - wm_metrics['wm_mean_B']) - \
                             (wm_after['wm_mean_A'] - wm_metrics['wm_mean_A'])
            
            results.append({
                'entangle_strength': strength,
                'seed': seed,
                'accuracy': acc,
                'separation_A': wm_metrics['separation_A'],
                'separation_B': wm_metrics['separation_B'],
                'entanglement_effect': entangle_effect,
                'B_survived': wm_after['wm_mean_B'] > 0.5
            })
    
    return results

def run_ablation_key_dim(config, seeds=[42, 2024]):
    """Ablation: Effect of key dimension"""
    results = []
    
    for key_dim in config.KEY_DIMS:
        for seed in seeds:
            set_seed(seed)
            
            X_train, y_train, X_test, y_test = load_fashion_mnist(
                config.N_TRAIN, config.N_TEST, seed
            )
            
            train_ds = TensorDataset(X_train, y_train)
            test_ds = TensorDataset(X_test, y_test)
            train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
            test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE)
            
            n_wm = int(len(train_ds) * config.WATERMARK_RATIO)
            wm_indices = np.random.choice(len(train_ds), n_wm, replace=False)
            clean_indices = [i for i in range(len(train_ds)) if i not in wm_indices]
            
            wm_ds = Subset(train_ds, wm_indices)
            clean_ds = Subset(train_ds, clean_indices[:n_wm])
            wm_loader = DataLoader(wm_ds, batch_size=config.BATCH_SIZE, shuffle=True)
            clean_loader = DataLoader(clean_ds, batch_size=config.BATCH_SIZE)
            
            model = EntangledDualWatermarkModel(
                config.MODEL_CONFIGS['Medium'],
                key_dim=key_dim,
                entangle_strength=config.ENTANGLE_STRENGTH,
                seed=seed
            ).to(device)
            
            train_task(model, train_loader, config.EPOCHS_TASK, config.LR_TASK)
            train_watermark(model, train_loader, wm_loader, config.EPOCHS_WATERMARK, config.LR_WATERMARK)
            
            acc = evaluate_accuracy(model, test_loader)
            wm_metrics, _ = evaluate_watermark(model, clean_loader, wm_loader)
            ortho = model.get_orthogonality_metrics()
            
            results.append({
                'key_dim': key_dim,
                'seed': seed,
                'accuracy': acc,
                'separation_A': wm_metrics['separation_A'],
                'separation_B': wm_metrics['separation_B'],
                'key_orthogonality': ortho['key_orthogonality'],
                'proj_correlation': ortho['projection_correlation']
            })
    
    return results

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    print_header("ENTANGLED DUAL WATERMARKS: COMPREHENSIVE EXPERIMENTAL STUDY")
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Device: {device}")
    print(f"PyTorch Version: {torch.__version__}")
    
    config = Config()
    
    # =========================================================================
    # SECTION 1: MAIN EXPERIMENTS (Multiple models × Multiple seeds)
    # =========================================================================
    print_header("SECTION 1: MAIN EXPERIMENTS")
    print(f"Models: {list(config.MODEL_CONFIGS.keys())}")
    print(f"Seeds: {config.SEEDS}")
    print(f"Total experiments: {len(config.MODEL_CONFIGS) * len(config.SEEDS)}")
    
    all_results = []
    
    for model_name in config.MODEL_CONFIGS.keys():
        print_subheader(f"Model: {model_name}")
        
        for seed in config.SEEDS:
            print(f"\n  Seed {seed}:", end=" ")
            t0 = time.time()
            
            try:
                results = run_single_experiment(config, model_name, seed, verbose=False)
                results['time'] = time.time() - t0
                all_results.append(results)
                print(f"✓ ({results['time']:.1f}s, Acc={results['task_acc_baseline']:.1%}, "
                      f"Sep_A={results['baseline_separation_A']:.3f}, Sep_B={results['baseline_separation_B']:.3f})")
            except Exception as e:
                print(f"✗ Error: {e}")
    
    # =========================================================================
    # SECTION 2: AGGREGATE RESULTS BY MODEL
    # =========================================================================
    print_header("SECTION 2: AGGREGATE RESULTS BY MODEL")
    
    for model_name in config.MODEL_CONFIGS.keys():
        model_results = [r for r in all_results if r['config'] == model_name]
        if not model_results:
            continue
        
        print_subheader(f"Model: {model_name} (n={len(model_results)})")
        
        # Baseline metrics
        print("\n  [BASELINE PERFORMANCE]")
        acc = [r['task_acc_baseline'] for r in model_results]
        sep_A = [r['baseline_separation_A'] for r in model_results]
        sep_B = [r['baseline_separation_B'] for r in model_results]
        eff_A = [r['baseline_effect_size_A'] for r in model_results]
        eff_B = [r['baseline_effect_size_B'] for r in model_results]
        
        print(f"  Task Accuracy:     {np.mean(acc):.4f} ± {np.std(acc):.4f}")
        print(f"  Separation A:      {np.mean(sep_A):.4f} ± {np.std(sep_A):.4f}")
        print(f"  Separation B:      {np.mean(sep_B):.4f} ± {np.std(sep_B):.4f}")
        print(f"  Effect Size A:     {np.mean(eff_A):.4f} ± {np.std(eff_A):.4f}")
        print(f"  Effect Size B:     {np.mean(eff_B):.4f} ± {np.std(eff_B):.4f}")
        
        # Orthogonality
        print("\n  [ORTHOGONALITY METRICS]")
        key_ortho = [r['ortho_key_orthogonality'] for r in model_results]
        proj_corr = [r['ortho_projection_correlation'] for r in model_results]
        print(f"  Key Orthogonality: {np.mean(key_ortho):.6f} ± {np.std(key_ortho):.6f}")
        print(f"  Proj Correlation:  {np.mean(proj_corr):.4f} ± {np.std(proj_corr):.4f}")
        
        # Entanglement effects
        print("\n  [ENTANGLEMENT EFFECTS]")
        ent_A = [r['entanglement_effect_A'] for r in model_results]
        ent_B = [r['entanglement_effect_B'] for r in model_results]
        print(f"  Attack A → Effect: {np.mean(ent_A):.4f} ± {np.std(ent_A):.4f} (positive=B preserved)")
        print(f"  Attack B → Effect: {np.mean(ent_B):.4f} ± {np.std(ent_B):.4f} (positive=A preserved)")
        
        # Attack robustness
        print("\n  [ATTACK ROBUSTNESS]")
        
        # Targeted A attack
        a_killed = [r['attack_A_wm_mean_A'] for r in model_results]
        b_after_a = [r['attack_A_wm_mean_B'] for r in model_results]
        print(f"  After Attack A:    A={np.mean(a_killed):.4f}±{np.std(a_killed):.4f}, "
              f"B={np.mean(b_after_a):.4f}±{np.std(b_after_a):.4f}")
        
        # Targeted B attack
        b_killed = [r['attack_B_wm_mean_B'] for r in model_results]
        a_after_b = [r['attack_B_wm_mean_A'] for r in model_results]
        print(f"  After Attack B:    A={np.mean(a_after_b):.4f}±{np.std(a_after_b):.4f}, "
              f"B={np.mean(b_killed):.4f}±{np.std(b_killed):.4f}")
        
        # Dual attack
        a_dual = [r['attack_dual_wm_mean_A'] for r in model_results]
        b_dual = [r['attack_dual_wm_mean_B'] for r in model_results]
        print(f"  After Dual Attack: A={np.mean(a_dual):.4f}±{np.std(a_dual):.4f}, "
              f"B={np.mean(b_dual):.4f}±{np.std(b_dual):.4f}")
        
        # Fine-tuning
        for epochs in [3, 5, 10]:
            a_ft = [r[f'attack_ft{epochs}_wm_mean_A'] for r in model_results]
            b_ft = [r[f'attack_ft{epochs}_wm_mean_B'] for r in model_results]
            acc_ft = [r[f'attack_ft{epochs}_acc'] for r in model_results]
            print(f"  After FT {epochs:2d}ep:     A={np.mean(a_ft):.4f}, B={np.mean(b_ft):.4f}, "
                  f"Acc={np.mean(acc_ft):.4f}")
        
        # Pruning
        for ratio in [30, 50, 70]:
            a_pr = [r[f'attack_prune{ratio}_wm_mean_A'] for r in model_results]
            b_pr = [r[f'attack_prune{ratio}_wm_mean_B'] for r in model_results]
            acc_pr = [r[f'attack_prune{ratio}_acc'] for r in model_results]
            print(f"  After Prune {ratio}%:   A={np.mean(a_pr):.4f}, B={np.mean(b_pr):.4f}, "
                  f"Acc={np.mean(acc_pr):.4f}")
    
    # =========================================================================
    # SECTION 3: STATISTICAL SIGNIFICANCE TESTS
    # =========================================================================
    print_header("SECTION 3: STATISTICAL SIGNIFICANCE TESTS")
    
    print("\n  [Watermark Detection Significance - All Models Combined]")
    all_sep_A = [r['baseline_separation_A'] for r in all_results]
    all_sep_B = [r['baseline_separation_B'] for r in all_results]
    
    # One-sample t-test against 0
    t_A, p_A = stats.ttest_1samp(all_sep_A, 0)
    t_B, p_B = stats.ttest_1samp(all_sep_B, 0)
    
    print(f"  Separation A > 0:  t={t_A:.4f}, p={format_pvalue(p_A)}")
    print(f"  Separation B > 0:  t={t_B:.4f}, p={format_pvalue(p_B)}")
    
    print("\n  [Entanglement Effect Significance]")
    all_ent_A = [r['entanglement_effect_A'] for r in all_results]
    t_ent, p_ent = stats.ttest_1samp(all_ent_A, 0)
    print(f"  Entanglement > 0:  t={t_ent:.4f}, p={format_pvalue(p_ent)}")
    print(f"  Mean Effect:       {np.mean(all_ent_A):.4f} ± {np.std(all_ent_A):.4f}")
    
    print("\n  [Survival Rate After Attacks]")
    for attack in ['attack_A', 'attack_B', 'attack_dual']:
        survive_A = sum(1 for r in all_results if r[f'{attack}_wm_mean_A'] > 0.5)
        survive_B = sum(1 for r in all_results if r[f'{attack}_wm_mean_B'] > 0.5)
        survive_any = sum(1 for r in all_results if r[f'{attack}_wm_mean_A'] > 0.5 or r[f'{attack}_wm_mean_B'] > 0.5)
        print(f"  {attack:15s}: A survives {survive_A}/{len(all_results)}, "
              f"B survives {survive_B}/{len(all_results)}, "
              f"Any survives {survive_any}/{len(all_results)}")
    
    # =========================================================================
    # SECTION 4: ABLATION STUDIES
    # =========================================================================
    print_header("SECTION 4: ABLATION STUDIES")
    
    # 4.1 Entanglement Strength
    print_subheader("4.1 Effect of Entanglement Strength")
    ablation_strength = run_ablation_entangle_strength(config, seeds=[42, 2024, 7])
    
    print(f"\n  {'Strength':>10} | {'Accuracy':>10} | {'Sep_A':>10} | {'Sep_B':>10} | {'Entangle':>10} | {'B_Surv':>8}")
    print("  " + "-"*70)
    
    for strength in config.ENTANGLE_STRENGTHS:
        s_results = [r for r in ablation_strength if r['entangle_strength'] == strength]
        acc = np.mean([r['accuracy'] for r in s_results])
        sep_a = np.mean([r['separation_A'] for r in s_results])
        sep_b = np.mean([r['separation_B'] for r in s_results])
        ent = np.mean([r['entanglement_effect'] for r in s_results])
        surv = sum(1 for r in s_results if r['B_survived']) / len(s_results)
        print(f"  {strength:>10.1f} | {acc:>10.4f} | {sep_a:>10.4f} | {sep_b:>10.4f} | {ent:>10.4f} | {surv:>8.1%}")
    
    # 4.2 Key Dimension
    print_subheader("4.2 Effect of Key Dimension")
    ablation_dim = run_ablation_key_dim(config, seeds=[42, 2024, 7])
    
    print(f"\n  {'Key Dim':>10} | {'Accuracy':>10} | {'Sep_A':>10} | {'Sep_B':>10} | {'Key Ortho':>10} | {'Proj Corr':>10}")
    print("  " + "-"*75)
    
    for dim in config.KEY_DIMS:
        d_results = [r for r in ablation_dim if r['key_dim'] == dim]
        acc = np.mean([r['accuracy'] for r in d_results])
        sep_a = np.mean([r['separation_A'] for r in d_results])
        sep_b = np.mean([r['separation_B'] for r in d_results])
        ortho = np.mean([r['key_orthogonality'] for r in d_results])
        corr = np.mean([r['proj_correlation'] for r in d_results])
        print(f"  {dim:>10d} | {acc:>10.4f} | {sep_a:>10.4f} | {sep_b:>10.4f} | {ortho:>10.6f} | {corr:>10.4f}")
    
    # =========================================================================
    # SECTION 5: DETAILED RESULTS TABLE (FOR PAPER)
    # =========================================================================
    print_header("SECTION 5: DETAILED RESULTS TABLE (PAPER-READY)")
    
    print("\n  TABLE 1: Baseline Watermark Detection Performance")
    print("  " + "="*100)
    print(f"  {'Model':>10} | {'Seed':>6} | {'Acc':>8} | {'WM_A':>8} | {'Clean_A':>8} | "
          f"{'Sep_A':>8} | {'WM_B':>8} | {'Clean_B':>8} | {'Sep_B':>8}")
    print("  " + "-"*100)
    
    for r in all_results:
        print(f"  {r['config']:>10} | {r['seed']:>6} | {r['task_acc_baseline']:>8.4f} | "
              f"{r['baseline_wm_mean_A']:>8.4f} | {r['baseline_clean_mean_A']:>8.4f} | "
              f"{r['baseline_separation_A']:>8.4f} | {r['baseline_wm_mean_B']:>8.4f} | "
              f"{r['baseline_clean_mean_B']:>8.4f} | {r['baseline_separation_B']:>8.4f}")
    
    print("\n  TABLE 2: Attack Robustness - Watermark Scores After Attack")
    print("  " + "="*110)
    print(f"  {'Model':>10} | {'Seed':>6} | {'Baseline_A':>10} | {'Baseline_B':>10} | "
          f"{'AttackA_A':>10} | {'AttackA_B':>10} | {'AttackB_A':>10} | {'AttackB_B':>10} | {'Dual_A':>8} | {'Dual_B':>8}")
    print("  " + "-"*110)
    
    for r in all_results:
        print(f"  {r['config']:>10} | {r['seed']:>6} | {r['baseline_wm_mean_A']:>10.4f} | "
              f"{r['baseline_wm_mean_B']:>10.4f} | {r['attack_A_wm_mean_A']:>10.4f} | "
              f"{r['attack_A_wm_mean_B']:>10.4f} | {r['attack_B_wm_mean_A']:>10.4f} | "
              f"{r['attack_B_wm_mean_B']:>10.4f} | {r['attack_dual_wm_mean_A']:>8.4f} | "
              f"{r['attack_dual_wm_mean_B']:>8.4f}")
    
    print("\n  TABLE 3: Entanglement Effects")
    print("  " + "="*80)
    print(f"  {'Model':>10} | {'Seed':>6} | {'A_before':>10} | {'A_after':>10} | "
          f"{'B_before':>10} | {'B_after':>10} | {'Entangle':>10} | {'B_Survive':>10}")
    print("  " + "-"*80)
    
    for r in all_results:
        b_survive = "YES" if r['attack_A_wm_mean_B'] > 0.5 else "NO"
        print(f"  {r['config']:>10} | {r['seed']:>6} | {r['baseline_wm_mean_A']:>10.4f} | "
              f"{r['attack_A_wm_mean_A']:>10.4f} | {r['baseline_wm_mean_B']:>10.4f} | "
              f"{r['attack_A_wm_mean_B']:>10.4f} | {r['entanglement_effect_A']:>10.4f} | {b_survive:>10}")
    
    print("\n  TABLE 4: Fine-Tuning Robustness")
    print("  " + "="*95)
    print(f"  {'Model':>10} | {'Seed':>6} | {'Baseline_A':>10} | {'Baseline_B':>10} | "
          f"{'FT3_A':>8} | {'FT3_B':>8} | {'FT5_A':>8} | {'FT5_B':>8} | {'FT10_A':>8} | {'FT10_B':>8}")
    print("  " + "-"*95)
    
    for r in all_results:
        print(f"  {r['config']:>10} | {r['seed']:>6} | {r['baseline_wm_mean_A']:>10.4f} | "
              f"{r['baseline_wm_mean_B']:>10.4f} | {r['attack_ft3_wm_mean_A']:>8.4f} | "
              f"{r['attack_ft3_wm_mean_B']:>8.4f} | {r['attack_ft5_wm_mean_A']:>8.4f} | "
              f"{r['attack_ft5_wm_mean_B']:>8.4f} | {r['attack_ft10_wm_mean_A']:>8.4f} | "
              f"{r['attack_ft10_wm_mean_B']:>8.4f}")
    
    print("\n  TABLE 5: Pruning Robustness")
    print("  " + "="*95)
    print(f"  {'Model':>10} | {'Seed':>6} | {'Baseline_A':>10} | {'Baseline_B':>10} | "
          f"{'P30_A':>8} | {'P30_B':>8} | {'P50_A':>8} | {'P50_B':>8} | {'P70_A':>8} | {'P70_B':>8}")
    print("  " + "-"*95)
    
    for r in all_results:
        print(f"  {r['config']:>10} | {r['seed']:>6} | {r['baseline_wm_mean_A']:>10.4f} | "
              f"{r['baseline_wm_mean_B']:>10.4f} | {r['attack_prune30_wm_mean_A']:>8.4f} | "
              f"{r['attack_prune30_wm_mean_B']:>8.4f} | {r['attack_prune50_wm_mean_A']:>8.4f} | "
              f"{r['attack_prune50_wm_mean_B']:>8.4f} | {r['attack_prune70_wm_mean_A']:>8.4f} | "
              f"{r['attack_prune70_wm_mean_B']:>8.4f}")
    
    # =========================================================================
    # SECTION 6: SUMMARY STATISTICS (FOR PAPER ABSTRACT/INTRO)
    # =========================================================================
    print_header("SECTION 6: SUMMARY STATISTICS")
    
    # Overall performance
    mean_acc = np.mean([r['task_acc_baseline'] for r in all_results])
    std_acc = np.std([r['task_acc_baseline'] for r in all_results])
    mean_sep_A = np.mean([r['baseline_separation_A'] for r in all_results])
    std_sep_A = np.std([r['baseline_separation_A'] for r in all_results])
    mean_sep_B = np.mean([r['baseline_separation_B'] for r in all_results])
    std_sep_B = np.std([r['baseline_separation_B'] for r in all_results])
    
    print(f"\n  OVERALL PERFORMANCE (n={len(all_results)} experiments)")
    print(f"  ────────────────────────────────────────────────────────")
    print(f"  Task Accuracy:           {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"  Watermark Separation A:  {mean_sep_A:.4f} ± {std_sep_A:.4f}")
    print(f"  Watermark Separation B:  {mean_sep_B:.4f} ± {std_sep_B:.4f}")
    
    # Entanglement effectiveness
    mean_ent = np.mean([r['entanglement_effect_A'] for r in all_results])
    std_ent = np.std([r['entanglement_effect_A'] for r in all_results])
    ent_positive_rate = sum(1 for r in all_results if r['entanglement_effect_A'] > 0) / len(all_results)
    b_survive_rate = sum(1 for r in all_results if r['attack_A_wm_mean_B'] > 0.5) / len(all_results)
    
    print(f"\n  ENTANGLEMENT EFFECTIVENESS")
    print(f"  ────────────────────────────────────────────────────────")
    print(f"  Entanglement Effect:     {mean_ent:.4f} ± {std_ent:.4f}")
    print(f"  Positive Effect Rate:    {ent_positive_rate:.1%}")
    print(f"  B Survival Rate:         {b_survive_rate:.1%}")
    
    # Attack robustness summary
    print(f"\n  ATTACK ROBUSTNESS SUMMARY")
    print(f"  ────────────────────────────────────────────────────────")
    
    attacks = [
        ('Targeted A', 'attack_A'),
        ('Targeted B', 'attack_B'),
        ('Dual Attack', 'attack_dual'),
        ('Fine-Tune 5ep', 'attack_ft5'),
        ('Prune 50%', 'attack_prune50'),
    ]
    
    for name, key in attacks:
        retain_A = np.mean([r[f'{key}_wm_mean_A'] / r['baseline_wm_mean_A'] for r in all_results if r['baseline_wm_mean_A'] > 0.1])
        retain_B = np.mean([r[f'{key}_wm_mean_B'] / r['baseline_wm_mean_B'] for r in all_results if r['baseline_wm_mean_B'] > 0.1])
        any_survive = sum(1 for r in all_results if r[f'{key}_wm_mean_A'] > 0.5 or r[f'{key}_wm_mean_B'] > 0.5) / len(all_results)
        print(f"  {name:20s}: A_retain={retain_A:.1%}, B_retain={retain_B:.1%}, Any_survive={any_survive:.1%}")
    
    # =========================================================================
    # SECTION 7: KEY CLAIMS FOR PAPER
    # =========================================================================
    print_header("SECTION 7: KEY CLAIMS FOR PAPER")
    
    print("""
  Based on the experimental results, the following claims are supported:

  CLAIM 1: DUAL WATERMARK EMBEDDING
  ─────────────────────────────────
  The proposed framework successfully embeds two statistically independent
  watermarks (A and B) with orthogonal keys.""")
    print(f"  - Mean Separation A: {mean_sep_A:.4f} (p{format_pvalue(p_A)})")
    print(f"  - Mean Separation B: {mean_sep_B:.4f} (p{format_pvalue(p_B)})")
    
    print("""
  CLAIM 2: ENTANGLEMENT TRAP MECHANISM
  ─────────────────────────────────────
  Attacking watermark A triggers the entanglement mechanism, preserving
  watermark B with high probability.""")
    print(f"  - Entanglement Effect: {mean_ent:.4f} ± {std_ent:.4f}")
    print(f"  - B Survival Rate when A Attacked: {b_survive_rate:.1%}")
    
    print("""
  CLAIM 3: REMOVAL IMPOSSIBILITY
  ──────────────────────────────
  Adversaries cannot remove both watermarks without significantly
  degrading model accuracy.""")
    any_survive_dual = sum(1 for r in all_results if r['attack_dual_wm_mean_A'] > 0.3 or r['attack_dual_wm_mean_B'] > 0.3) / len(all_results)
    print(f"  - Survival rate after dual attack: {any_survive_dual:.1%}")
    
    print("""
  CLAIM 4: ROBUSTNESS TO STANDARD ATTACKS
  ───────────────────────────────────────
  Watermarks survive fine-tuning and pruning attacks.""")
    ft5_survive = sum(1 for r in all_results if r['attack_ft5_wm_mean_A'] > 0.3 or r['attack_ft5_wm_mean_B'] > 0.3) / len(all_results)
    prune50_survive = sum(1 for r in all_results if r['attack_prune50_wm_mean_A'] > 0.3 or r['attack_prune50_wm_mean_B'] > 0.3) / len(all_results)
    print(f"  - Survival after 5-epoch fine-tuning: {ft5_survive:.1%}")
    print(f"  - Survival after 50% pruning: {prune50_survive:.1%}")
    
    # =========================================================================
    # SECTION 8: TIMING ANALYSIS
    # =========================================================================
    print_header("SECTION 8: COMPUTATIONAL ANALYSIS")
    
    times = [r['time'] for r in all_results]
    print(f"\n  Total experiments:     {len(all_results)}")
    print(f"  Total runtime:         {sum(times):.1f} seconds")
    print(f"  Mean per experiment:   {np.mean(times):.1f} ± {np.std(times):.1f} seconds")
    print(f"  Min/Max:               {min(times):.1f} / {max(times):.1f} seconds")
    
    for model in config.MODEL_CONFIGS.keys():
        m_times = [r['time'] for r in all_results if r['config'] == model]
        print(f"  {model:>10s} model:      {np.mean(m_times):.1f} ± {np.std(m_times):.1f} seconds")
    
    # =========================================================================
    # FINAL SUMMARY
    # =========================================================================
    print_header("EXPERIMENT COMPLETE")
    print(f"""
  ✓ {len(all_results)} main experiments completed
  ✓ {len(ablation_strength)} entanglement strength ablation experiments
  ✓ {len(ablation_dim)} key dimension ablation experiments
  ✓ All statistical tests performed
  ✓ Paper-ready tables generated
  
  Key Findings:
  ─────────────
  1. Dual watermarks successfully embedded (Sep_A={mean_sep_A:.3f}, Sep_B={mean_sep_B:.3f})
  2. Entanglement trap effective ({ent_positive_rate:.0%} positive effect rate)
  3. B survives when A attacked ({b_survive_rate:.0%} survival rate)
  4. Robust to standard attacks (FT: {ft5_survive:.0%}, Prune: {prune50_survive:.0%})
  
  Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
  """)

# =============================================================================
# RUN
# =============================================================================

if __name__ == "__main__":
    main()


  ENTANGLED DUAL WATERMARKS: COMPREHENSIVE EXPERIMENTAL STUDY
Timestamp: 2025-12-26 08:47:52
Device: cuda
PyTorch Version: 2.6.0+cu124

  SECTION 1: MAIN EXPERIMENTS
Models: ['Small', 'Medium', 'Large']
Seeds: [42, 2024, 7, 123, 999]
Total experiments: 15

--------------------------------------------------------------------------------
  Model: Small
--------------------------------------------------------------------------------

  Seed 42: ✓ (9.7s, Acc=77.5%, Sep_A=0.221, Sep_B=0.235)

  Seed 2024: ✓ (9.7s, Acc=83.5%, Sep_A=0.137, Sep_B=0.136)

  Seed 7: ✓ (9.8s, Acc=81.5%, Sep_A=0.146, Sep_B=0.150)

  Seed 123: ✓ (9.7s, Acc=86.5%, Sep_A=0.164, Sep_B=0.169)

  Seed 999: ✓ (9.7s, Acc=87.0%, Sep_A=0.226, Sep_B=0.225)

--------------------------------------------------------------------------------
  Model: Medium
--------------------------------------------------------------------------------

  Seed 42: ✓ (9.4s, Acc=81.0%, Sep_A=0.295, Sep_B=0.307)

  Seed 2024: ✓ (9.9s, Acc=87.5%, S